In [1]:
import os

from pybel import readfile
from kalasanty.net import UNet
from tfbio.data import Featurizer

import logging
logging.getLogger('tensorflow').setLevel(logging.FATAL)

/Users/gaopeijun/opt/anaconda3/envs/kalasanty_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gaopeijun/opt/anaconda3/envs/kalasanty_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gaopeijun/opt/anaconda3/envs/kalasanty_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

# Load trained model

Do not modify this cell - model will not work with different parametrs than it was trained with.
If you use the network trained by us, you must use the same `scale`, `max_dist` and `featurizer`.

In [2]:
model_path = 'data/model_scpdb2017.hdf'

model = UNet.load_model(model_path, scale=0.5, max_dist=35, featurizer=Featurizer(save_molecule_codes=False))

/Users/gaopeijun/opt/anaconda3/envs/kalasanty_env/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


# Load the protein

You can ran Kalasanty on one of the provided examples (see below) or you can upload your molecule. For optimal performance, make sure that your molecule is protonized and charged.

To upload your molecule, get back to the main window and click "Upload" in the top right part of the screen.

Available test examples:

In [3]:
! ls tests/datasets/*/*/*protein.mol2

tests/datasets/pdbbind/2oc2/2oc2_protein.mol2
tests/datasets/pdbbind/2qfo/2qfo_protein.mol2
tests/datasets/pdbbind/2ybu/2ybu_protein.mol2
tests/datasets/pdbbind/3arx/3arx_protein.mol2
tests/datasets/pdbbind/4hfp/4hfp_protein.mol2
tests/datasets/scpdb/2oc2_1/protein.mol2
tests/datasets/scpdb/2qfo_1/protein.mol2
tests/datasets/scpdb/2qfo_2/protein.mol2
tests/datasets/scpdb/2ybu_8/protein.mol2
tests/datasets/scpdb/3arx_1/protein.mol2
tests/datasets/scpdb/4hfp_2/protein.mol2


Place path to the protein in the cell below. If you uploaded molecule in a different format than mol2, specify it as the first argument.

In [9]:
mol = next(readfile('mol2', 'tests/datasets/scpdb/2ybu_8/protein.mol2'))

In [83]:
mol.atoms

 ...]

In [77]:
import json

In [85]:
mol.atoms[3].atomicnum

7

In [99]:
def encode_residue(res):
    #encoding_types = ['Acthely_factors']
    encoding_type='Acthely_factors'
    residue_types = ['ALA','CYS','ASP','GLU','PHE','GLY','HIS','ILE','LYS','LEU','MET','ASN','PRO','GLN','ARG','SER','THR','VAL','TRP','TYR','XAA']
    res = res.residue.name[:3]
    with open("/Users/gaopeijun/kalasanty/%s.json" % encoding_type, 'r') as load_f:
        encoding = json.load(load_f)
    encoding_data = []
    if res not in residue_types:
        res = "XAA"
    encoded_res = encoding[res]
    #encoding_data.append({res: encoding[res]})                
    #return encoding_data
    return encoded_res

In [100]:
res = mol.atoms[3]

In [105]:
import numpy as np
res1 = encode_residue(res)
a=np.array(res1).flatten()
a.shape

(5,)

# Predict pockets

Save amino-acids forming the pockets as separate mol2 files

In [27]:
pockets = model.predict_pocket_atoms(mol)
for i, pocket in enumerate(pockets):
    pocket.write('mol2', 'pocket%i.mol2' % i, overwrite=True)

Save pocket probability density as .cube file

In [28]:
density, origin, step = model.pocket_density_from_mol(mol)

In [29]:
model.save_density_as_cube(density, origin, step, fname='pockets.cube')

Get back to the main page and download `pocket*.mol2` files, `pockets.cube` file, and optionally input file you used (e.g. `tests/datasets/scpdb/2qfo_1/protein.mol2`) and open them in your favourite molecular modeling software.